# COMP30027 Machine Learning Project 2

## Description of text features

This notebook describes the pre-computed text features provided for Project 2. **You do not need to recompute the features yourself for this assignment** -- this information is just for your reference. However, feel free to experiment with different text features if you are interested. If you do want to try generating your own text features, some things to keep in mind:
- There are many different decisions you can make throughout the feature design process, from the text preprocessing to the size of the output vectors. There's no guarantee that the defaults we chose will produce the best possible text features for this classification task, so feel free to experiment with different settings.
- These features must be trained using a training corpus. Generally, the training corpus should not include validation samples, but for the purposes of this assignment we have used the entire non-test set (training+validation) as the training corpus, to allow you to experiment with different validation sets. If you recompute the text features as part of your own model, you should exclude validation samples and compute them on training samples only. For example, if you do N-fold cross-validation, this means generating N sets of features for N different training-validation splits.

In [1]:
import numpy as np
import pandas as pd

# read text
# for DEMONSTRATION PURPOSES, the entire training set will be used to train the models and also as a test set
x_train_original = pd.read_csv(r"book_rating_train.csv", index_col = False, delimiter = ',', header=0)
# use recipe name as an example
train_corpus_name = x_train_original['Name']
test_name = x_train_original['Name']

In [2]:

# x_train_original.loc[:,"rating_label" == True]


测试数据是否对齐

In [3]:
length = len(x_train_original)
for i in range(length):
    print(len(x_train_original.iloc[i,:]))
    if (len(x_train_original.iloc[i,:]) != 10):
        # print(len((x_train_original.iloc[i,:])))
        print("ha")
    

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

忽略部分数据 没有label的直接删除

数据对其

## Count vectorizer

A count vectorizer converts documents to vectors which represent word counts. Each column in the output represents a different word and the values indicate the number of times that word appeared in the document. The overall size of a count vector matrix can be quite large (the number of columns is the total number of different words used across all documents in a corpus), but most entries in the matrix are zero (each document contains only a few of all the possible words). Therefore, it is most efficient to represent the count vectors as a sparse matrix.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# preprocess text and compute counts

vocab_name = CountVectorizer(stop_words='english').fit(train_corpus_name)

# generate counts for a new set of documents
x_train_name = vocab_name.transform(train_corpus_name)
x_test_name = vocab_name.transform(test_name)

# check the number of words in vocabulary
# print(x_test_name)
print(vocab_name.vocabulary_)
# check the shape of sparse matrix
print(x_train_name.shape)

## doc2vec

doc2vec methods are an extension of word2vec. word2vec maps words to a high-dimensional vector space in such a way that words which appear in similar contexts will be close together in the space. doc2vec does a similar embedding for multi-word passages. The doc2vec (or Paragraph Vector) method was introduced by:

**Le & Mikolov (2014)** Distributed Representations of Sentences and Documents<br>
https://arxiv.org/pdf/1405.4053v2.pdf

The implementation of doc2vec used for this project is from gensim and documented here:<br>
https://radimrehurek.com/gensim/models/doc2vec.html

The size of the output vector is a free parameter. Most implemementations use around 100-300 dimensions, but the best size depends on the problem you're trying to solve with the embeddings and the number of training samples, so you may wish to try different vector sizes. We provided doc2vec features for Name (vec_size = 100), Authors (vec_size = 20) and Description (vec_size = 100). The vectors themselves represent directions in a high-dimensional concept space; the columns do not represent specific words or phrases. Values in the vector are continuous real numbers and can be negative.

In [ ]:
import gensim

# size of the output vector
vec_size = 100

# function to preprocess and tokenize text
def tokenize_corpus(txt, tokens_only=False):
    for i, line in enumerate(txt):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

# tokenize a training corpus
corpus_name = list(tokenize_corpus(train_corpus_name))

# train doc2vec on the training corpus
model = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=40)
model.build_vocab(corpus_name)
model.train(corpus_name, total_examples=model.corpus_count, epochs=model.epochs)

# tokenize new documents
doc = list(tokenize_corpus(test_name, tokens_only=True))

# generate embeddings for the new documents
x_test_name = np.zeros((len(doc),vec_size))
for i in range(len(doc)):
    x_test_name[i,:] = model.infer_vector(doc[i])
    
# check the shape of doc_emb
print(x_test_name.shape)

print(x_test_name[0, :])

# x_test_name[:, :]

(23063, 100)
[ 0.09194423 -0.15658826 -0.01671327  0.16211915 -0.02784856 -0.06803759
  0.19201237 -0.15129901  0.15996583  0.00505785 -0.01628309 -0.29460675
 -0.14412256  0.4145014  -0.05053574  0.13099663  0.08230288 -0.00634621
 -0.18229097  0.0288527   0.21220189  0.08361877 -0.09053814  0.06700892
  0.03652943 -0.00798776 -0.10323533 -0.04589715 -0.18327434 -0.1584737
  0.30714944  0.21252392  0.05665923 -0.11217452 -0.24837664 -0.02586861
  0.10087958  0.04762876  0.0742768  -0.02975656 -0.18492043 -0.05994615
  0.02093898 -0.10476821  0.06144366  0.09394655 -0.01805695  0.03127558
  0.06857584  0.10069478  0.16664699 -0.14130051 -0.18697108 -0.23770967
  0.00590979  0.11813767 -0.06923162  0.1114826  -0.01824936 -0.12340856
  0.19756781 -0.11678986 -0.08383447 -0.191021    0.19147594  0.15524703
 -0.04476053 -0.25787643 -0.24746311 -0.10866081 -0.12308696  0.06945705
 -0.35623327 -0.25740325  0.04015918 -0.09166218  0.24028666  0.03893771
  0.09033373 -0.07049097  0.05663535  0

In [ ]:


# # 读取原始数据
x_train_original.drop(columns= ['Description'])
insert = pd.DataFrame(x_test_name);
# x_train_original
# pd.set_option('display.max_columns', None) # 显示所有列 
# pd.set_option('display.max_rows', None) # 显示所有行 
# pd.set_option('display.width', None) # 显示完整宽度 
# pd.set_option('display.max_colwidth', -1) # 显示完整单元格内容
# df_concat = pd.concat([x_train_original, insert], axis=1, ignore_index=True)
df_concat = pd.concat([x_train_original, insert], axis=1)
df_concat.iloc[0]

                                                                   Name  \
0  Best of Dr Jean: Reading & Writing                                     
1  Here All Dwell Free                                                    
2  Boomer's Big Surprise                                                  
3  I'll Go and Do More: Annie Dodge Wauneka, Navajo Leader and Activist   
4  Us                                                                     

                  Authors  PublishYear  PublishMonth  PublishDay  \
0  Jean R. Feldman         2005         6             1            
1  Gertrud Mueller Nelson  1991         10            1            
2  Constance W. McGeorge   2005         3             31           
3  Carolyn Niethammer      2004         9             1            
4  Richard       Mason     2005         7             7            

            Publisher Language  pagesNumber  rating_label  
0  Teaching Resources  NaN      48           4.0           
1  DoubleDay        

Name            Best of Dr Jean: Reading & Writing                                                                                                                                                                                                                                                                      
Authors         Jean R. Feldman                                                                                                                                                                                                                                                                                         
PublishYear     2005                                                                                                                                                                                                                                                                                                    
PublishMonth    6                                            

获取label

In [ ]:
# x_test = df_concat.iloc[:,9]
# x_test
# df_concat.head()
label = df_concat.iloc[:,9]
process_data = df_concat.drop(columns = ['rating_label','Description'])
process_data.head()

try_data = process_data.drop(columns= ['Name','Authors','Publisher','Language'])

stacking model

In [ ]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

np.random.seed(30027)

class StackingClassifier():

    def __init__(self, classifiers, metaclassifier):
        self.classifiers = classifiers
        self.metaclassifier = metaclassifier

    def fit(self, X, y):
        for clf in self.classifiers:
            clf.fit(X, y)
        X_meta = self._predict_base(X) #X_meta is the output (y_hat) of the base classifiers
        self.metaclassifier.fit(X_meta, y) #output of the base classifiers is the input for the meta classifier
    
    def _predict_base(self, X):
        yhats = []
        for clf in self.classifiers:
            yhat = clf.predict_proba(X)
            yhats.append(yhat)
        yhats = np.concatenate(yhats, axis=1)
        assert yhats.shape[0] == X.shape[0] # check that the number of rows yhats matches the number of rows in the input data X
        return yhats
    
    def predict(self, X):
        X_meta = self._predict_base(X)     
        yhat = self.metaclassifier.predict(X_meta)
        return yhat
    def score(self, X, y):
        yhat = self.predict(X)
        return accuracy_score(y, yhat)
    


classifiers = [DummyClassifier(strategy='most_frequent'),
                LogisticRegression(),
                KNeighborsClassifier(),
                GaussianNB(),
                ]
titles = ['Zero_R',
          'Logistic Regression',
          'KNN',
          'Gaussian NB',  
          ]

meta_classifier_lr = LogisticRegression()
stacker_lr = StackingClassifier(classifiers, meta_classifier_lr)

meta_classifier_dt = DecisionTreeClassifier()
stacker_dt = StackingClassifier(classifiers, meta_classifier_dt)


stackin出现的问题 无法统筹连续和非连续同时出现的情况，我的处理方式是把那些feature 直接不要

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(try_data, label, test_size=0.33, random_state=30027)
stacker_lr.fit(X_train, y_train)
print('\nStacker Accuracy (Logistic Regression):', stacker_lr.score(X_test, y_test))

c:\Users\13065\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\13065\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\13065\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(



Stacker Accuracy (Logistic Regression): 0.6468269609775326


c:\Users\13065\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
